In [59]:
import pandas as pd
import json
import numpy as np

## Chargons les données

In [60]:
path_data = '../data/clean_data/application_train.csv'
variables = ['AMT_REQ_CREDIT_BUREAU_YEAR', 'HOUR_APPR_PROCESS_START', 'AMT_ANNUITY', 'AMT_CREDIT','EXT_SOURCE_3',
            'EXT_SOURCE_2', 'CODE_GENDER', 'FLAG_PHONE', 'AMT_GOODS_PRICE', 'FLAG_OWN_CAR', 'NAME_FAMILY_STATUS', "TARGET"]
df_train = pd.read_csv(path_data)
df_train = df_train[variables]
df_train = df_train[df_train['CODE_GENDER'] != 'XNA']
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 307507 entries, 0 to 307510
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   AMT_REQ_CREDIT_BUREAU_YEAR  265988 non-null  float64
 1   HOUR_APPR_PROCESS_START     307507 non-null  int64  
 2   AMT_ANNUITY                 307495 non-null  float64
 3   AMT_CREDIT                  307507 non-null  float64
 4   EXT_SOURCE_3                246542 non-null  float64
 5   EXT_SOURCE_2                306847 non-null  float64
 6   CODE_GENDER                 307507 non-null  object 
 7   FLAG_PHONE                  307507 non-null  int64  
 8   AMT_GOODS_PRICE             307229 non-null  float64
 9   FLAG_OWN_CAR                307507 non-null  object 
 10  NAME_FAMILY_STATUS          307507 non-null  object 
 11  TARGET                      307507 non-null  int64  
dtypes: float64(6), int64(3), object(3)
memory usage: 30.5+ MB


In [61]:
df_train.sample(10)

,AMT_REQ_CREDIT_BUREAU_YEAR,HOUR_APPR_PROCESS_START,AMT_ANNUITY,AMT_CREDIT,EXT_SOURCE_3,EXT_SOURCE_2,CODE_GENDER,FLAG_PHONE,AMT_GOODS_PRICE,FLAG_OWN_CAR,NAME_FAMILY_STATUS,TARGET
274117,1.0,16,32058.0,1161000.0,0.177704,0.595091,F,0,1161000.0,N,Single / not married,0
223347,0.0,17,10215.0,284400.0,0.835777,0.675403,F,1,225000.0,N,Married,0
247632,NaN,12,14193.0,391500.0,NaN,0.448777,F,0,391500.0,N,Separated,0
120755,3.0,8,9675.0,204768.0,0.269286,0.014359,F,0,162000.0,N,Married,0
291059,0.0,9,21748.5,273636.0,0.583238,0.223096,M,0,247500.0,Y,Married,0
148049,4.0,15,38533.5,1190434.5,0.303146,0.688370,F,1,1039500.0,N,Married,0
210771,0.0,13,19867.5,675000.0,0.709189,0.668704,F,1,675000.0,Y,Married,0
142517,0.0,4,37800.0,1288350.0,0.595456,0.678125,F,0,1125000.0,Y,Married,0
59471,5.0,10,53550.0,2025000.0,0.728141,0.603896,F,1,2025000.0,N,Married,0
226593,5.0,14,28372.5,970380.0,0.691021,0.695483,M,1,810000.0,Y,Married,0


In [62]:
# chargemment des logs
with open("../logs/api_logs.jsonl", "r") as f:
    for line in f:
        records.append(json.loads(line))

logs_df = pd.DataFrame(records)


In [63]:
# analyse opérationelle ( latence )
latency_df = logs_df[logs_df["latency_ms"].notna()]

latency_df["latency_ms"].describe()

count      60.000000
mean      140.545833
std       422.074477
min         1.020000
25%         5.080000
50%        17.095000
75%        20.922500
max      1528.480000
Name: latency_ms, dtype: float64

In [64]:
slow_requests = latency_df[latency_df["latency_ms"] > 1000]
slow_requests.head()

,AMT_REQ_CREDIT_BUREAU_YEAR,HOUR_APPR_PROCESS_START,AMT_ANNUITY,AMT_CREDIT,EXT_SOURCE_3,EXT_SOURCE_2,CODE_GENDER,FLAG_PHONE,AMT_GOODS_PRICE,FLAG_OWN_CAR,...,prediction,probability,latency_ms,status_code,timestamp,request_id,path,method,request_payload,model_output
0,3.0,16.0,27733.5,948582.0,0.771362,0.644347,M,1.0,679500.0,Y,...,0.0,0.431281,1528.48,200,1.769626e+09,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1528.48,200,1.769626e+09,8bd7393c-3a7d-417b-92e4-227ed2e78ea7,/predict,POST,"{'AMT_REQ_CREDIT_BUREAU_YEAR': 3.0, 'HOUR_APPR...","{'prediction': 0, 'probability': 0.43128063129..."
24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1528.48,200,1.769626e+09,8bd7393c-3a7d-417b-92e4-227ed2e78ea7,/predict,POST,"{'AMT_REQ_CREDIT_BUREAU_YEAR': 3.0, 'HOUR_APPR...","{'prediction': 0, 'probability': 0.43128063129..."
37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1528.48,200,1.769626e+09,8bd7393c-3a7d-417b-92e4-227ed2e78ea7,/predict,POST,"{'AMT_REQ_CREDIT_BUREAU_YEAR': 3.0, 'HOUR_APPR...","{'prediction': 0, 'probability': 0.43128063129..."
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1528.48,200,1.769626e+09,8bd7393c-3a7d-417b-92e4-227ed2e78ea7,/predict,POST,"{'AMT_REQ_CREDIT_BUREAU_YEAR': 3.0, 'HOUR_APPR...","{'prediction': 0, 'probability': 0.43128063129..."


In [65]:
#taux d'erreurs
error_rate = (logs_df["status_code"] >= 400).mean()
print(error_rate)

0.13333333333333333


In [66]:
# Extraction des features depuis request_payload
predict_logs = logs_df[
    (logs_df["path"] == "/predict") &
    (logs_df["status_code"] == 200)
]

features_df = pd.json_normalize(predict_logs["request_payload"])
features_df.head()

,AMT_REQ_CREDIT_BUREAU_YEAR,HOUR_APPR_PROCESS_START,AMT_ANNUITY,AMT_CREDIT,EXT_SOURCE_3,EXT_SOURCE_2,CODE_GENDER,FLAG_PHONE,AMT_GOODS_PRICE,FLAG_OWN_CAR,NAME_FAMILY_STATUS
0,3.0,16,27733.5,948582.0,0.771362,0.644347,M,1,679500.0,Y,Married
1,1.0,10,14751.0,269550.0,0.000527,0.653998,M,0,225000.0,Y,Married
2,3.0,11,47511.0,447768.0,0.239226,0.125744,F,0,405000.0,N,Civil marriage
3,3.0,19,40063.5,1006920.0,0.749022,0.703540,F,1,900000.0,N,Married
4,3.0,16,10728.0,135000.0,0.441836,0.541243,F,1,135000.0,N,Married


In [67]:
# netoyage du data frame
numerical_cols = [
    "AMT_REQ_CREDIT_BUREAU_YEAR",
    "HOUR_APPR_PROCESS_START",
    "AMT_ANNUITY",
    "AMT_CREDIT",
    "EXT_SOURCE_3",
    "EXT_SOURCE_2",
    "AMT_GOODS_PRICE",
    "FLAG_PHONE",
]

categorical_cols = [
    "CODE_GENDER",
    "FLAG_OWN_CAR",
    "NAME_FAMILY_STATUS",
]

for col in numerical_cols:
    features_df[col] = pd.to_numeric(features_df[col], errors="coerce")

features_df["CODE_GENDER"] = features_df["CODE_GENDER"].replace("XNA", np.nan)


## Construction du rapport EVIDENTLY

## DATA DRIFT

In [68]:
import os
import numpy as np
from evidently import Report, DataDefinition, ColumnType, Dataset
from evidently.presets import DataDriftPreset

In [69]:
definition = DataDefinition(
    numerical_columns=numerical_cols,
    categorical_columns=categorical_cols
)

In [70]:
reference_dataset = Dataset.from_pandas(
    df_train[numerical_cols + categorical_cols],
    data_definition=definition
)

current_dataset = Dataset.from_pandas(
    features_df[numerical_cols + categorical_cols],
    data_definition=definition
)


In [71]:
report = Report(metrics=[DataDriftPreset()])

my_eval = report.run(reference_dataset, current_dataset)

c:\Users\laelo\Desktop\credit_scoring\.venv\Lib\site-packages\scipy\stats\_stats_py.py:7400: RuntimeWarning:

divide by zero encountered in divide

c:\Users\laelo\Desktop\credit_scoring\.venv\Lib\site-packages\scipy\stats\_stats_py.py:7400: RuntimeWarning:

divide by zero encountered in divide

c:\Users\laelo\Desktop\credit_scoring\.venv\Lib\site-packages\scipy\stats\_stats_py.py:7400: RuntimeWarning:

divide by zero encountered in divide



In [72]:
os.makedirs("../reports", exist_ok=True)
my_eval.save_html("../reports/data_drift.html")

## Rapport Opérationnel (latence, erreur)

In [77]:
# Préparer les colonnes opérationnelles
ops_df = logs_df[["latency_ms", "status_code"]].copy()
ops_df["latency_ms"] = pd.to_numeric(ops_df["latency_ms"], errors="coerce")
ops_df["status_code"] = pd.to_numeric(ops_df["status_code"], errors="coerce")

In [78]:
# Référence vs courant (ops)
# Même logique que pour le data drift : reference : période stable (ex: premiers logs) et current : période récente
ops_ref = ops_df.iloc[:5]
ops_curr = ops_df.iloc[5:]

In [79]:
ops_definition = DataDefinition(numerical_columns=["latency_ms", "status_code"])
ops_ref_dataset = Dataset.from_pandas(ops_ref, data_definition=ops_definition)
ops_curr_dataset = Dataset.from_pandas(ops_curr, data_definition=ops_definition)

In [80]:
ops_report = Report(metrics=[DataDriftPreset()])

# Lancer l’analyse
my_report = ops_report.run(reference_data=ops_ref_dataset, current_data=ops_curr_dataset)

# Sauvegarder en HTML
my_report.save_html("../reports/ops_monitoring.html")

c:\Users\laelo\Desktop\credit_scoring\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning:

invalid value encountered in divide

c:\Users\laelo\Desktop\credit_scoring\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning:

invalid value encountered in divide

c:\Users\laelo\Desktop\credit_scoring\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning:

invalid value encountered in divide

c:\Users\laelo\Desktop\credit_scoring\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning:

invalid value encountered in divide

